In [1]:
########################################################################################
######################### QUEL SCREEN TWITTER EST A ANALYSER ? #########################
########################################################################################
#MonScreenName = 'AdetFrederic'
#Langue = 'french'
MonScreenName = 'KimKardashian'
Langue = 'english'

In [2]:
# IMPORT PANDAS
import pandas as pd
# IMPORT TWEEPY
import tweepy
# IMPORT JSON RE & STRING ...
import json
import re
import string
import operator 
from collections import Counter
# IMPORT EMOJI & EMOTICON'S
import emoji
# IMPORT NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# Gestion configparser
import configparser

In [3]:
# Gestion de la configuration
config = configparser.ConfigParser()
config.read('../Configuration.conf')

['../Configuration.conf']

In [4]:
# Obtention de mes identifiants Twitter
ma_consumer_key = config.get('Twitter', 'consumer_key')
ma_consumer_secret = config.get('Twitter', 'consumer_secret')
ma_Oauth_token = config.get('Twitter', 'Oauth_token')
ma_Oauth_token_secret = config.get('Twitter', 'Oauth_token_secret')

In [5]:
# IDENTIFICATION DE MON API TWITTER
auth = tweepy.OAuthHandler(ma_consumer_key, ma_consumer_secret)
auth.set_access_token(ma_Oauth_token, ma_Oauth_token_secret)
api = tweepy.API(auth)

In [6]:
# FONCTION OBTENTION TWEETS DANS DATAFRAME
def MaRechercheTweets(ScreenName):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = ScreenName, count = 200)
    #save most recent tweets
    alltweets.extend(new_tweets)
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = ScreenName, count = 200, max_id = oldest)
        #save most recent tweets
        alltweets.extend(new_tweets)
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print ('... %s Tweets chargés' % (len(alltweets)))
    #transform the tweepy tweets into a 2D array
    #outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.coordinates] for tweet in alltweets]
    return outtweets

In [7]:
# FONCTION DE TRAITEMENT DU TEXTE DES TWEETS
def MonTraitementTweets(MonTweet, lowercase=False):
    Tokens = tokenize(MonTweet) # FONCTION(S) Tokenize
    TokensEmojis = [Token for Token in ''.join(c for c in MonTweet if c in emoji.UNICODE_EMOJI)] # Emoji's
    TokensEmojisSens = [emoji.demojize(Token) for Token in TokensEmojis] # Emoji's (Sens)
    TokensURLs = [Token for Token in Tokens if Token.startswith('http')] # URL's
    TokensHastags = [Token for Token in Tokens if Token.startswith('#')] # Hastag's
    TokensMentions = [Token for Token in Tokens if Token.startswith('@')] # Mention's
    TokensRestants = [Token for Token in Tokens if Token not in TokensEmojis
                                                and Token not in TokensURLs
                                                and Token not in TokensHastags
                                                and Token not in TokensMentions]
    TokensRestants = [Token.lower() for Token in TokensRestants if Token not in Punctuation] # Punctuation 
    TokensRestants = [Token.lower() for Token in TokensRestants if Token not in StopWords] # Stopwords                
    return pd.Series([TokensRestants, TokensEmojis, TokensEmojisSens, TokensURLs, TokensHastags, TokensMentions])

In [8]:
# FONCTION TOKENIZE
def tokenize(s):
    return tokens_re.findall(s)

In [9]:
# GESTION DES EMOTICON's POUR TOKENIZE
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [10]:
# GESTION REGEX POUR TOKENIZE
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

In [11]:
# GESTION DE LA PONCTUATION
Punctuation = list(string.punctuation) + ['“', '…', '”']

In [12]:
# GESTION DES STOPWORDS
if Langue == 'english':
    StopWords = stopwords.words(Langue) + ['rt', 'via']             # Pour l'anglais
if Langue == 'french':
    StopWords = stopwords.words(Langue) + ['rt', 'via', 'les']      # Pour le français

In [13]:
TweetsCherches = MaRechercheTweets(MonScreenName)
TweetsCherches_df = pd.DataFrame(TweetsCherches, columns = ['Tweet_Id', 'Tweet_Date', 'Tweet_Texte', 'Tweet_GeoLoc'])

... 400 Tweets chargés
... 600 Tweets chargés
... 797 Tweets chargés
... 996 Tweets chargés
... 1196 Tweets chargés
... 1396 Tweets chargés
... 1595 Tweets chargés
... 1795 Tweets chargés
... 1995 Tweets chargés
... 2194 Tweets chargés
... 2394 Tweets chargés
... 2594 Tweets chargés
... 2794 Tweets chargés
... 2994 Tweets chargés
... 3194 Tweets chargés
... 3244 Tweets chargés
... 3244 Tweets chargés


In [14]:
# TRAITEMENT DU TEXTE DES TWEETS
TweetsCherches_df[['Tokens', 'Emojis', 'SensEmojis', 'URLs', 'Hastags', 'Mentions']] = TweetsCherches_df.apply(lambda x: MonTraitementTweets(x['Tweet_Texte']), axis = 1)
TweetsCherches_df

,Tweet_Id,Tweet_Date,Tweet_Texte,Tweet_GeoLoc,Tokens,Emojis,SensEmojis,URLs,Hastags,Mentions
0,1120908392296411137,2019-04-24 04:32:40,RT @ScottDisick: https://t.co/2u6A9M0UQd,None,[],[],[],[https://t.co/2u6A9M0UQd],[],[@ScottDisick]
1,1120820740859650049,2019-04-23 22:44:23,RT @enews: Get ready for #FlipItLikeDisick. 🏠 ...,None,"[get, ready]",[🏠],[:house:],[https://t.co/4kVe8crcxT],[#FlipItLikeDisick],[@enews]
2,1120736359503683584,2019-04-23 17:09:05,Mason filmed the first video lol,None,"[mason, filmed, first, video, lol]",[],[],[],[],[]
3,1120736352281092097,2019-04-23 17:09:03,🕷😱 https://t.co/ymDvmHrLDn,None,[],"[🕷, 😱]","[:spider:, :face_screaming_in_fear:]",[https://t.co/ymDvmHrLDn],[],[]
4,1120736042644955136,2019-04-23 17:07:49,🕷 https://t.co/ca8ND7tMam,None,[],[🕷],[:spider:],[https://t.co/ca8ND7tMam],[],[]
5,1120728825061470209,2019-04-23 16:39:08,RT @KrisJenner: Scott’s new show “Flip It Like...,None,"[scott, ’, new, show, flip, like, disick, prem...",[],[],[https://t.co/Tu9QwdIo5I],[#FlipItLikeDisick],"[@KrisJenner, @ScottDisick]"
6,1120728047315906560,2019-04-23 16:36:03,RT @TeamKanyeDaily: SUNDAY SERVICE MERCH \n\nh...,None,"[sunday, service, merch]",[],[],"[https://t.co/ZUko9NhxhV, https://t.co/AwsR2E2...",[],[@TeamKanyeDaily]
7,1120581425709764608,2019-04-23 06:53:25,RT @ajzfern: Did you know that the $1Billion +...,None,"[know, 1, billion, donated, notre, dame, would...",[],[],[],[],[@ajzfern]
8,1120540331005620226,2019-04-23 04:10:08,"RT @TMZ: Over 50,000 People Attended Kanye Wes...",None,"[50,000, people, attended, kanye, west's, coac...",[],[],[https://t.co/hhej4Wjnir],[],[@TMZ]
9,1120468788212850688,2019-04-22 23:25:51,RT @hypebeastmusic: A bird's eye view of @kany...,None,"[bird's, eye, view, courtesy]",[],[],[https://t.co/4RyMkZjFWB],[#SundayService],"[@hypebeastmusic, @kanyewest, @coachella]"


In [15]:
# ANALYSE DE L'ENSEMBLE DES TWEETS (MOTS UTILES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Tokens'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES TWEETS (MOTS UTILES)')
print(Comptage.most_common(20))

********** ANALYSE DES TWEETS (MOTS UTILES)
[('’', 576), ('amp', 318), ('pm', 252), ('shop', 232), ('pst', 224), ('️', 221), ('12', 219), ('new', 205), ('today', 179), ('available', 165), ('get', 141), ('love', 136), ('lip', 127), ('collection', 125), ('classic', 125), ('happy', 123), ('tomorrow', 120), ('day', 111), ('pop-up', 106), ('crème', 103)]


In [16]:
# ANALYSE DE L'ENSEMBLE DES MENTIONS (PERSONNES RE-TWEETEES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Mentions'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES MENTIONS (PERSONNES RE-TWEETEES)')
print(Comptage.most_common(20))

********** ANALYSE DES MENTIONS (PERSONNES RE-TWEETEES)
[('@kkwbeauty', 464), ('@KimKardashian', 165), ('@KKWFRAGRANCE', 131), ('@KKWMAFIA', 61), ('@AsyaKardashian', 41), ('@MakeupByMario', 36), ('@WestfieldCC', 31), ('@khloekardashian', 27), ('@MUNCHKiN_XO', 26), ('@KrisJenner', 25), ('@kanyewest', 25), ('@southcoastplaza', 24), ('@TeamKanyeDaily', 22), ('@voguemagazine', 20), ('@kuwthewests', 19), ('@', 19), ('@narcicismo', 19), ('@KKWBEAUTY', 17), ('@kimkardashian', 15), ('@itsKELSEYtho', 14)]


In [17]:
# ANALYSE DE L'ENSEMBLE DES HASHTAGS (REFERENCES)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['Hastags'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES HASHTAGS (REFERENCES)')
print(Comptage.most_common(20))

********** ANALYSE DES HASHTAGS (REFERENCES)
[('#KKWBEAUTY', 166), ('#KUWTK', 81), ('#ConcealBakeBrighten', 72), ('#KKWBODY', 69), ('#KKWxMARIO', 64), ('#KKWXMARIO', 58), ('#KKWFRAGRANCE', 36), ('#', 28), ('#FlashingLights', 25), ('#UltralightBeams', 22), ('#KIMOJIPERFUME', 20), ('#GlamMasters', 17), ('#KKWXARGENIS', 13), ('#KKWHEARTS', 10), ('#MarchForOurLives', 10), ('#KKWxARGENIS', 10), ('#YouKiddinMe', 9), ('#YEEZY', 7), ('#CyberMonday', 7), ('#BlackFriday', 7)]


In [18]:
# ANALYSE DE L'ENSEMBLE DES EMOJIS (LEUR SENS)
Comptage = Counter()
TsLesTokens = []
for i in range (0, len(TweetsCherches_df)):
    TsLesTokens.extend(TweetsCherches_df['SensEmojis'][i])
Comptage.update(TsLesTokens)
print('********** ANALYSE DES EMOJIS (LEUR SENS)')
print(Comptage.most_common(20))

********** ANALYSE DES EMOJIS (LEUR SENS)
[(':smiling_face_with_heart-eyes:', 177), (':kiss_mark:', 150), (':red_heart:', 145), (':sparkles:', 126), (':face_with_tears_of_joy:', 125), (':fire:', 107), (':two_hearts:', 103), (':blue_heart:', 52), (':cherry_blossom:', 52), (':clapping_hands:', 50), (':peach:', 39), (':growing_heart:', 35), (':cherries:', 30), (':folded_hands:', 29), (':smiling_face_with_3_hearts:', 28), (':purple_heart:', 27), (':black_heart:', 19), (':speech_balloon:', 19), (':female_sign:', 17), (':face_screaming_in_fear:', 16)]
